import all the relevant libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as stats
import io
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split


upload the db file

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#helper functions cross val sensitivity specifity

In [ ]:
def calcCrossValidation_Matrix(model, df, predictors, outcome):

  total_tp = 0
  total_tn = 0
  total_fp = 0
  total_fn = 0

  # Perform k-fold cross-validation with 10 folds
  kf = KFold(n_splits=57)
  accuracy = []
  for train, test in kf.split(df):
      # print("1")

      # Filter training data
      train_predictors = (df[predictors].iloc[train, :])
      # The target we're using to train the algorithm.
      train_target = df[outcome].iloc[train]
      # print("1")

      test_predictors = (df[predictors].iloc[test, :])
      # The target we're using to train the algorithm.
      test_target = df[outcome].iloc[test]
      # print("1")


      # Training the algorithm using the predictors and target.
      model.fit(train_predictors, train_target)
      # print("fit done")
      predictions = model.predict(test_predictors)

      # cm = confusion_matrix(test_target, predictions)
      # print(cm)
      # print("1")
      # print("test_target")
      test_target = test_target.to_numpy()
      # print(test_target)
      # np.array(df[outcome].values.tolist())
      # print(type(test_target))
      # print("predictions")
      # print(predictions)
      # print(type(predictions))

      tn, fp, fn, tp = perf_measure(test_target, predictions)
      # print("1")

      total_tp += tp
      total_tn += tn
      total_fp += fp
      total_fn += fn

  # print(total_tn, total_fp, total_fn, total_tp)
  return total_tn, total_fp, total_fn, total_tp

In [ ]:
def calcCrossValidation_Matrix_extend_train(model, df, predictors, outcome,df_extend_train):

  total_tp = 0
  total_tn = 0
  total_fp = 0
  total_fn = 0

  # Perform k-fold cross-validation with 10 folds
  kf = KFold(n_splits=57)
  accuracy = []
  for train, test in kf.split(df):
      train_predictors = (df[predictors].iloc[train, :])
      # print("train_predictors before concat")
      # print(train_predictors)
      train_target = df[outcome].iloc[train]
      # print("train_target before concat")
      # print(train_target)
      # print("shape")
      # print(train_predictors.shape)

      train_predictors = pd.concat([train_predictors, df_extend_train[predictors]])
      # print("train_predictors after concat")
      # print(train_predictors)
      train_target = pd.concat([train_target, df_extend_train[outcome]])
      # print("train_target after concat")
      # print(train_target)
      # print("shape")
      # print(train_predictors.shape)
      test_predictors = (df[predictors].iloc[test, :])
      test_target = df[outcome].iloc[test]
      model.fit(train_predictors, train_target)
      predictions = model.predict(test_predictors)
      test_target = test_target.to_numpy()

      tn, fp, fn, tp = perf_measure(test_target, predictions)

      total_tp += tp
      total_tn += tn
      total_fp += fp
      total_fn += fn

  return total_tn, total_fp, total_fn, total_tp

In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1
    # print(TN, FP, FN, TP)
    return(TN, FP, FN, TP)

In [ ]:
def calc_specificity(tn, fp, fn, tp):
  return tn / (tn +fp)

In [ ]:
def calc_sensitivity(tn, fp, fn, tp):
  return tp / (tp +fn)

In [ ]:
def calcCrossValidation_RMSE(model, df, predictors, outcome):


  # Perform leave one out cross-validation
  kf = KFold(n_splits=df.shape[0])
  accuracy = []
  total_RMSE=0
  for train, test in kf.split(df):
      train_predictors = (df[predictors].iloc[train, :])
      train_target = df[outcome].iloc[train]
      test_predictors = (df[predictors].iloc[test, :])
      test_target = df[outcome].iloc[test]
      model.fit(train_predictors, train_target)
      predictions = model.predict(test_predictors)
      rmse = sqrt(mean_squared_error(test_target, predictions))  
      total_RMSE += rmse

  return total_RMSE/df.shape[0]

In [ ]:
def calcCrossValidation_RMSE_extend_train(model, df, predictors, outcome,df_extend_train):


  # Perform leave one out cross-validation
  kf = KFold(n_splits=df.shape[0])
  accuracy = []
  total_RMSE=0
  for train, test in kf.split(df):
      train_predictors = (df[predictors].iloc[train, :])
      train_target = df[outcome].iloc[train]

      train_predictors = pd.concat([train_predictors, df_extend_train[predictors]])
      train_target = pd.concat([train_target, df_extend_train[outcome]])

      test_predictors = (df[predictors].iloc[test, :])
      test_target = df[outcome].iloc[test]
      model.fit(train_predictors, train_target)
      predictions = model.predict(test_predictors)
      rmse = sqrt(mean_squared_error(test_target, predictions))  
      total_RMSE += rmse

  return total_RMSE/df.shape[0]     
     


# code for automation experiments

In [ ]:
def run_experiment(df, outcome_var, model_type, predictor_var, removed_features, df_experiments,df_extend_train):
  model = choose_experiment_model(df, outcome_var, model_type, predictor_var, removed_features, df_experiments)
  # print("model chose")
  if (model_type=='Random Forest Classifier' or model_type=='Logistic Regression'):
    calculations = add_sensitivity_specifity(model, df, predictor_var, outcome_var, model_type, removed_features, df_experiments,df_extend_train)
    # print("calculation done")

  else:
    calculations = add_rmse(model, df, predictor_var, outcome_var, model_type, removed_features, df_experiments,df_extend_train)
  experiment_results = update_experiment_in_results_df(calculations, removed_features, model_type, df_experiments)
  return experiment_results



In [ ]:
def choose_experiment_model(df, outcome_var, model_type, predictor_var, removed_features, df_experiments):
  if (model_type=='Random Forest Classifier'):
    return RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=SEED)
    # return add_sensitivity_specifity(model, df, predictor_var, outcome_var, model_type, removed_features, df_experiments)
  elif (model_type=='Logistic Regression'):
    return LogisticRegression(max_iter = 4000,random_state=SEED)
    # return add_sensitivity_specifity(model, df, predictor_var, outcome_var, model_type, removed_features, df_experiments)
  else:
    return RandomForestRegressor(n_estimators = 10, random_state=SEED)
    # return add_rmse(model, df, predictor_var, outcome_var, model_type, removed_features, df_experiments)

In [ ]:
def add_sensitivity_specifity(model, df, predictor_var, outcome_var, model_type, removed_features, df_experiments,df_extend_train):
  # print("enter add sens")
  # tn, fp, fn, tp = calcCrossValidation_Matrix(model, df, predictor_var, outcome_var)
  tn, fp, fn, tp = calcCrossValidation_Matrix_extend_train(model, df, predictor_var, outcome_var,df_extend_train)
  
  # print("calcCrossValidation_Matrix done")
  specificity = calc_specificity(tn, fp, fn, tp)
  sensitivity = calc_sensitivity(tn, fp, fn, tp)
  specifity_sensitivity_list = [specificity, sensitivity]
  return specifity_sensitivity_list

In [ ]:
def add_rmse(model, df, predictor_var, outcome_var, model_type, removed_features, df_experiments,df_extend_train):
  # rmse = calcCrossValidation_RMSE(model, df, predictor_var, outcome_var)
  rmse = calcCrossValidation_RMSE_extend_train(model, df, predictor_var, outcome_var,df_extend_train)
  
  rmse_list = [rmse]
  return rmse_list

In [ ]:
def update_experiment_in_results_df(results_list, removed_features, model_type, df_experiments):
  if (len(results_list)==2):
    new_record = {'Model': model_type,'Removed Features': removed_features,'Specificity': results_list[0], 'Sensitivity':results_list[1], 'RMSE': ''}
  else:
    new_record = {'Model': model_type,'Removed Features': removed_features,'Specificity': '', 'Sensitivity':'', 'RMSE':results_list[0]}
  df_experiments = df_experiments.append(new_record, ignore_index=True)
  return df_experiments
  

In [ ]:
def experiments_RF_LR(df_origin_data,features_list, features_names,df_experiments,df_extend_train):
  df_experiments = run_experiment(df_origin_data, OUTCOME_MS_TYPE, RFCLASSIFIER, features_list, features_names, df_experiments,df_extend_train)
  df_experiments = run_experiment(df_origin_data, OUTCOME_MS_TYPE, LR, features_list, features_names, df_experiments,df_extend_train)
  df_experiments = run_experiment(df_origin_data, OUTCOME_EDSS_POST_10_YEARS, RFREGRESSOR, features_list, features_names, df_experiments,df_extend_train)
  return df_experiments

#Defines experiment configuration -

In [ ]:
#Livnat

#lists of selected features

ALL_FEATURES = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M','oligoclonal_band_POS']
#REMOVE ONE FEATURE
NO_TIGIT =  ['TIM3_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M','oligoclonal_band_POS']
NO_LAG3 =['TIM3_RQ', 'TIGIT_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M','oligoclonal_band_POS']
NO_LYMPH_CELLS = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','PMN_Cells','gender_M','oligoclonal_band_POS']
NO_AGE =['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M','oligoclonal_band_POS']
NO_PMN_CELLS = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','gender_M','oligoclonal_band_POS']
NO_OCB =['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M']
NO_MRI_LESION = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Lymph_Cells','PMN_Cells','gender_M','oligoclonal_band_POS']
NO_GENDER = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','oligoclonal_band_POS']
NO_EDSS_AT_DIAGNOSIS = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M','oligoclonal_band_POS']
NO_TIM3 = ['TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M','oligoclonal_band_POS']

#REMOVE TWO FEATURES
NO_TIGIT_NO_LAG3 = ['TIM3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M','oligoclonal_band_POS']
NO_TIGIT_NO_LYMPH_CELLS = ['TIM3_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','PMN_Cells','gender_M','oligoclonal_band_POS']
NO_TIGIT_NO_AGE = ['TIM3_RQ', 'LAG3_RQ', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M','oligoclonal_band_POS']
NO_TIGIT_NO_PMN_CELLS = ['TIM3_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','gender_M','oligoclonal_band_POS']
NO_TIGIT_NO_OCB = ['TIM3_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M']
NO_TIGIT_NO_MRI_LESION = ['TIM3_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis','Lymph_Cells','PMN_Cells','gender_M','oligoclonal_band_POS']

NO_LAG3_NO_LYMPH_CELLS = ['TIM3_RQ', 'TIGIT_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','PMN_Cells','gender_M','oligoclonal_band_POS']
NO_LAG3_NO_AGE = ['TIM3_RQ', 'TIGIT_RQ', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M','oligoclonal_band_POS']
NO_LAG3_NO_PMN_CELLS = ['TIM3_RQ', 'TIGIT_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','gender_M','oligoclonal_band_POS']
NO_LAG3_NO_OCB = ['TIM3_RQ', 'TIGIT_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M']
NO_LAG3_NO_MRI_LESION = ['TIM3_RQ', 'TIGIT_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis','Lymph_Cells','PMN_Cells','gender_M','oligoclonal_band_POS']

NO_LYMPH_CELLS_NO_AGE = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','PMN_Cells','gender_M','oligoclonal_band_POS']
NO_LYMPH_CELLS_NO_PMN_CELLS = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','gender_M','oligoclonal_band_POS']
NO_LYMPH_CELLS_NO_OCB = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','PMN_Cells','gender_M']
NO_LYMPH_CELLS_NO_MRI_LESION = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis','PMN_Cells','gender_M','oligoclonal_band_POS']

#REMOVE THREE FEATURES
NO_TIGIT_NO_LAG3_NO_LYMPH_CELLS = ['TIM3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','PMN_Cells','gender_M','oligoclonal_band_POS']

#REMOVE FOUR FEATURES
NO_TIGIT_NO_LAG3_NO_LYMPH_CELLS_NO_AGE = ['TIM3_RQ', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','PMN_Cells','gender_M','oligoclonal_band_POS']





#lists of removed features as String 

ALL_FEATURES_STR = "ALL FEATURES"
#REMOVE ONE FEATURE
NO_TIGIT_STR =  "NO TIGIT"
NO_LAG3_STR = "NO LAG3"
NO_LYMPH_CELLS_STR = "NO LYMPH CELLS"
NO_AGE_STR = "NO AGE"
NO_PMN_CELLS_STR = "NO PMN CELLS"
NO_OCB_STR ="NO OCB"
NO_MRI_LESION_STR ="NO MRI LESION"
NO_GENDER_STR = "NO GENDER"
NO_EDSS_AT_DIAGNOSIS_STR = "NO EDSS AT DIAGNOSIS"
NO_TIM3_STR = "NO TIM3"

#REMOVE TWO FEATURES
NO_TIGIT_NO_LAG3_STR = "NO TIGIT, LAG3"
NO_TIGIT_NO_LYMPH_CELLS_STR = "NO TIGIT, LYMPH CELLS"
NO_TIGIT_NO_AGE_STR = "NO TIGIT, AGE"
NO_TIGIT_NO_PMN_CELLS_STR = "NO TIGIT, PMN CELLS"
NO_TIGIT_NO_OCB_STR = "NO TIGIT, OCB"
NO_TIGIT_NO_MRI_LESION_STR = "NO TIGIT, MRI LESION"

NO_LAG3_NO_LYMPH_CELLS_STR = "NO_LAG3, LYMPH_CELLS"
NO_LAG3_NO_AGE_STR = "NO_LAG3, AGE"
NO_LAG3_NO_PMN_CELLS_STR = "NO_LAG3, PMN_CELLS"
NO_LAG3_NO_OCB_STR = "NO_LAG3, OCB"
NO_LAG3_NO_MRI_LESION_STR = "NO_LAG3, MRI_LESION"

NO_LYMPH_CELLS_NO_AGE_STR = "NO_LYMPH_CELLS, AGE"
NO_LYMPH_CELLS_NO_PMN_CELLS_STR = "NO_LYMPH_CELLS, PMN_CELLS"
NO_LYMPH_CELLS_NO_OCB_STR = "NO_LYMPH_CELLS, OCB"
NO_LYMPH_CELLS_NO_MRI_LESION_STR = "NO_LYMPH_CELLS, MRI_LESION"


#REMOVE THREE FEATURES
NO_TIGIT_NO_LAG3_NO_LYMPH_CELLS_STR = "NO_TIGIT, LAG3, LYMPH_CELLS"

#REMOVE FOUR FEATURES
NO_TIGIT_NO_LAG3_NO_LYMPH_CELLS_NO_AGE_STR = "NO_TIGIT, LAG3, LYMPH_CELLS, AGE"





#lists of models
RFCLASSIFIER = "Random Forest Classifier"
LR = "Logistic Regression"
RFREGRESSOR = "Random Forest Regressor"


#lists of outcome
OUTCOME_MS_TYPE = "diagnosis_SPMS"
OUTCOME_EDSS_POST_10_YEARS = "EDSS_post_10_years"

#const seed all runs
SEED = 42


#Read Michal data

read the file and create data frame

In [ ]:
df_michal=pd.read_excel("/content/gdrive/MyDrive/Colab Notebooks/MS_Notebooks/Michal Cohort post AV 14-04-2021 for Nadav.xlsx", header=None, skiprows=1, names=['sample_code','Diagnosis date', 'diagnosis','gender','age_at_diagnosis', 'primary_EDSS_at_diagnosis', '1_years_EDSS_from_sample_date' , '5_years_from_sample', 'EDSS_post_10_years' ,'oligoclonal_band','Lymph_Cells','PMN_Cells','Cells', 'num_of_medicine' ,'relapse_rate_since_blood_sample', 'MRI_lesion_mass'])
# Print the first 10 rows of the data set
print(df_michal.head(10))
# Get summary of numerical variables
print(df_michal.describe())

In [ ]:
df_michal

In [ ]:
count_samples = df_michal.shape[0]
count_samples

84

In [ ]:
missing_data = pd.DataFrame({'total_missing': df_michal.isnull().sum(), 'perc_missing': (df_michal.isnull().sum()/count_samples)*100})
missing_data

,total_missing,perc_missing
sample_code,0,0.000000
Diagnosis date,0,0.000000
diagnosis,0,0.000000
gender,0,0.000000
age_at_diagnosis,0,0.000000
primary_EDSS_at_diagnosis,0,0.000000
1_years_EDSS_from_sample_date,0,0.000000
5_years_from_sample,0,0.000000
EDSS_post_10_years,0,0.000000
oligoclonal_band,13,15.476190


remove white space before and after the word:

In [ ]:
df_michal = df_michal.applymap(lambda x: x.strip() if type(x)==str else x)

Fix string values to upperCase

In [ ]:
df_michal['gender'] = df_michal['gender'].str.upper()
df_michal['diagnosis'] = df_michal['diagnosis'].str.upper()
df_michal['oligoclonal_band'] = df_michal['oligoclonal_band'].str.upper()


global variable for number of rows

In [ ]:
count_row = df_michal.shape[0]
count_row

84

replace 'uk' and 'ND' to 'Nan'

In [ ]:
df_michal = df_michal.replace({'UK': np.nan, 'uk': np.nan, 'ND': np.nan, 'N/A' : np.nan, '?' : np.nan, 'X': np.nan, 'MG RRMS' : 'RRMS', '2012*' : '2012', 'N\A': np.nan, '*' :  np.nan})

In [ ]:
df_michal.to_pickle("/content/gdrive/My Drive/Colab Notebooks/MS_Notebooks/MichalWithoutImputation.pkl")

In [ ]:
df_michal.columns

Index(['sample_code', 'Diagnosis date', 'diagnosis', 'gender',
       'age_at_diagnosis', 'primary_EDSS_at_diagnosis',
       '1_years_EDSS_from_sample_date', '5_years_from_sample',
       'EDSS_post_10_years', 'oligoclonal_band', 'Lymph_Cells', 'PMN_Cells',
       'Cells', 'num_of_medicine', 'relapse_rate_since_blood_sample',
       'MRI_lesion_mass'],
      dtype='object')

In [ ]:
df_oligoclonal_band = pd.get_dummies(df_michal['oligoclonal_band'])
df_gender = pd.get_dummies(df_michal['gender'])
df_diagnosis = pd.get_dummies(df_michal['diagnosis'])

df_michal = df_michal.merge(df_oligoclonal_band, left_index=True, right_index=True)
df_michal = df_michal.merge(df_gender, left_index=True, right_index=True)
df_michal = df_michal.merge(df_diagnosis, left_index=True, right_index=True)

df_michal = df_michal.rename(columns = {'POS': 'oligoclonal_band_POS','SPMS' : 'diagnosis_SPMS', 'M': 'gender_M'}, inplace = False)
# predictor_var = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
# outcome_var = 'diagnosis_SPMS'

In [ ]:
df_michal.to_pickle("/content/gdrive/My Drive/Colab Notebooks/MS_Notebooks/MichalWithoutImputation_include_dummies.pkl")

In [ ]:
columns = ['diagnosis', 'gender', 'age_at_diagnosis', 'primary_EDSS_at_diagnosis', '1_years_EDSS_from_sample_date', '5_years_from_sample',
       'EDSS_post_10_years', 'oligoclonal_band', 'Lymph_Cells', 'PMN_Cells', 'Cells', 'MRI_lesion_mass']

In [ ]:
df_michal.columns

Index(['sample_code', 'Diagnosis date', 'diagnosis', 'gender',
       'age_at_diagnosis', 'primary_EDSS_at_diagnosis',
       '1_years_EDSS_from_sample_date', '5_years_from_sample',
       'EDSS_post_10_years', 'oligoclonal_band', 'Lymph_Cells', 'PMN_Cells',
       'Cells', 'num_of_medicine', 'relapse_rate_since_blood_sample',
       'MRI_lesion_mass', 'NEG', 'oligoclonal_band_POS', 'F', 'gender_M',
       'RRMS', 'diagnosis_SPMS'],
      dtype='object')

In [ ]:
def min_max_normalize_features(dataframe, numeric_list):
    for num_feature in numeric_list:
        dataframe[num_feature] = (dataframe[num_feature] - dataframe[num_feature].min()) / (
                dataframe[num_feature].max() - dataframe[num_feature].min())
    return dataframe

In [ ]:
# normalized_features = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ', 'age_at_diagnosis','primary_EDSS_at_diagnosis', '1_years_EDSS_from_sample_date', 'MRI_lesion_mass', 'Lymph_Cells', 'PMN_Cells', 'Cells']
normalized_features = ['age_at_diagnosis', 'MRI_lesion_mass', 'Lymph_Cells', 'PMN_Cells', 'Cells']

df_michal = min_max_normalize_features(df_michal, normalized_features)
df_michal

#Read Livnat data

In [ ]:
df_livnat_all_patients = pd.read_pickle("/content/gdrive/My Drive/Colab Notebooks/MS_Notebooks/df_Livnat_all_patients.pkl")
df_livnat_all_patients

In [ ]:
normalized_features = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ', 'age_at_diagnosis', 'MRI_lesion_mass', 'Lymph_Cells', 'PMN_Cells', 'Cells']

df_livnat_all_patients = min_max_normalize_features(df_livnat_all_patients, normalized_features)
df_livnat_all_patients



In [ ]:
df_livnat_all_patients.columns

Index(['sample_code', 'TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ', 'age_at_diagnosis',
       'primary_EDSS_at_diagnosis', '1_years_EDSS_from_sample_date',
       '5_years_from_sample', 'EDSS_post_10_years', 'MRI_lesion_mass',
       'Lymph_Cells', 'PMN_Cells', 'Cells', 'gender_M', 'oligoclonal_band_POS',
       'diagnosis_SPMS'],
      dtype='object')

In [ ]:
predictor_var = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
X_train, X_test, y_train, y_test = train_test_split(df_livnat_all_patients[predictor_var], df_livnat_all_patients[outcome_var], random_state=42)

#Imputation

In [ ]:
df_livnat_all_patients.columns

Index(['sample_code', 'TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ', 'age_at_diagnosis',
       'primary_EDSS_at_diagnosis', '1_years_EDSS_from_sample_date',
       '5_years_from_sample', 'EDSS_post_10_years', 'MRI_lesion_mass',
       'Lymph_Cells', 'PMN_Cells', 'Cells', 'gender_M', 'oligoclonal_band_POS',
       'diagnosis_SPMS'],
      dtype='object')

In [ ]:
df_michal.columns

Index(['sample_code', 'Diagnosis date', 'diagnosis', 'gender',
       'age_at_diagnosis', 'primary_EDSS_at_diagnosis',
       '1_years_EDSS_from_sample_date', '5_years_from_sample',
       'EDSS_post_10_years', 'oligoclonal_band', 'Lymph_Cells', 'PMN_Cells',
       'Cells', 'num_of_medicine', 'relapse_rate_since_blood_sample',
       'MRI_lesion_mass', 'NEG', 'oligoclonal_band_POS', 'F', 'gender_M',
       'RRMS', 'diagnosis_SPMS'],
      dtype='object')

In [ ]:
 df_michal["LAG3_RQ"] = np.nan
 df_michal["TIM3_RQ"] = np.nan
 df_michal["TIGIT_RQ"] = np.nan


df_livnat_all_patients_copy = df_livnat_all_patients.copy()
df_michal_copy = df_michal.copy()
common_features = ['sample_code','TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ', 'age_at_diagnosis', 'primary_EDSS_at_diagnosis', '1_years_EDSS_from_sample_date', 'gender_M', 'oligoclonal_band_POS', 'Lymph_Cells', 'PMN_Cells', 'Cells', 'MRI_lesion_mass', '5_years_from_sample',
       'EDSS_post_10_years', 'diagnosis_SPMS']

df_livnat_all_patients_copy= df_livnat_all_patients_copy[common_features]
df_michal_copy = df_michal_copy[common_features]
df_michal_livnat = pd.concat([df_livnat_all_patients_copy, df_michal_copy])
df_michal_livnat.reset_index(drop=True)


In [ ]:
to_fill = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ', 'age_at_diagnosis', 'primary_EDSS_at_diagnosis', '1_years_EDSS_from_sample_date', 'gender_M', 'oligoclonal_band_POS', 'Lymph_Cells', 'PMN_Cells', 'Cells', 'MRI_lesion_mass','5_years_from_sample','EDSS_post_10_years','diagnosis_SPMS']

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
def imputation_fill(predictors):
  imputer = IterativeImputer(random_state=SEED)
  imputer.fit(predictors)
  predictors_transform = imputer.transform(predictors)
  return imputer, predictors_transform

model_imputation, X_train_transform = imputation_fill(df_michal_livnat[to_fill])
df_michal_copy_transform = model_imputation.transform(df_michal_copy[to_fill]);

#return data to df
df_michal_copy_transform = pd.DataFrame(df_michal_copy_transform, columns=to_fill)
# df_michal_copy_transform['5_years_from_sample'] = df_michal_copy['5_years_from_sample']
# df_michal_copy_transform['EDSS_post_10_years'] = df_michal_copy['EDSS_post_10_years']
# df_michal_copy_transform['diagnosis_SPMS'] = df_michal_copy['diagnosis_SPMS']
df_michal_copy_transform

#Random Forest - train test split without new samples

In [ ]:
model_no_addition_data = RandomForestClassifier(random_state=1)
model_no_addition_data.fit(X_train,y_train)
y_predicted = model_no_addition_data.predict(X_test)
#evaluate the model


## Running all experiments

In [ ]:



# df_experiments_michal = pd.DataFrame(columns=['Model','Removed Features','Specificity', 'Sensitivity', 'RMSE']) 

# df_experiments_michal = experiments_RF_LR(df_michal,ALL_FEATURES, ALL_FEATURES_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_TIGIT, NO_TIGIT_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal ,NO_LAG3, NO_LAG3_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_LYMPH_CELLS, NO_LYMPH_CELLS_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_AGE, NO_AGE_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_PMN_CELLS, NO_PMN_CELLS_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_OCB, NO_OCB_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_MRI_LESION, NO_MRI_LESION_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_GENDER, NO_GENDER_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_EDSS_AT_DIAGNOSIS, NO_EDSS_AT_DIAGNOSIS_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_TIM3, NO_TIM3_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_TIGIT_NO_LAG3, NO_TIGIT_NO_LAG3_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_TIGIT_NO_LYMPH_CELLS, NO_TIGIT_NO_LYMPH_CELLS_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_TIGIT_NO_AGE, NO_TIGIT_NO_AGE_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_TIGIT_NO_PMN_CELLS, NO_TIGIT_NO_PMN_CELLS_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_TIGIT_NO_OCB, NO_TIGIT_NO_OCB_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_TIGIT_NO_MRI_LESION, NO_TIGIT_NO_MRI_LESION_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_LAG3_NO_LYMPH_CELLS, NO_LAG3_NO_LYMPH_CELLS_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_LAG3_NO_AGE, NO_LAG3_NO_AGE_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_LAG3_NO_PMN_CELLS, NO_LAG3_NO_PMN_CELLS_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_LAG3_NO_OCB, NO_LAG3_NO_OCB_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_LAG3_NO_MRI_LESION, NO_LAG3_NO_MRI_LESION_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_LYMPH_CELLS_NO_AGE, NO_LYMPH_CELLS_NO_AGE_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_LYMPH_CELLS_NO_PMN_CELLS, NO_LYMPH_CELLS_NO_PMN_CELLS_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_LYMPH_CELLS_NO_OCB, NO_LYMPH_CELLS_NO_OCB_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_LYMPH_CELLS_NO_MRI_LESION, NO_LYMPH_CELLS_NO_MRI_LESION_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_TIGIT_NO_LAG3_NO_LYMPH_CELLS, NO_TIGIT_NO_LAG3_NO_LYMPH_CELLS_STR,df_experiments_michal)
# df_experiments_michal = experiments_RF_LR(df_michal,NO_TIGIT_NO_LAG3_NO_LYMPH_CELLS_NO_AGE, NO_TIGIT_NO_LAG3_NO_LYMPH_CELLS_NO_AGE_STR,df_experiments_michal)

In [ ]:
df_experiments = pd.DataFrame(columns=['Model','Removed Features','Specificity', 'Sensitivity', 'RMSE']) 

df_experiments = experiments_RF_LR(df_livnat_all_patients,ALL_FEATURES, ALL_FEATURES_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_TIGIT, NO_TIGIT_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients ,NO_LAG3, NO_LAG3_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_LYMPH_CELLS, NO_LYMPH_CELLS_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_AGE, NO_AGE_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_PMN_CELLS, NO_PMN_CELLS_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_OCB, NO_OCB_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_MRI_LESION, NO_MRI_LESION_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_GENDER, NO_GENDER_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_EDSS_AT_DIAGNOSIS, NO_EDSS_AT_DIAGNOSIS_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_TIM3, NO_TIM3_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_TIGIT_NO_LAG3, NO_TIGIT_NO_LAG3_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_TIGIT_NO_LYMPH_CELLS, NO_TIGIT_NO_LYMPH_CELLS_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_TIGIT_NO_AGE, NO_TIGIT_NO_AGE_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_TIGIT_NO_PMN_CELLS, NO_TIGIT_NO_PMN_CELLS_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_TIGIT_NO_OCB, NO_TIGIT_NO_OCB_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_TIGIT_NO_MRI_LESION, NO_TIGIT_NO_MRI_LESION_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_LAG3_NO_LYMPH_CELLS, NO_LAG3_NO_LYMPH_CELLS_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_LAG3_NO_AGE, NO_LAG3_NO_AGE_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_LAG3_NO_PMN_CELLS, NO_LAG3_NO_PMN_CELLS_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_LAG3_NO_OCB, NO_LAG3_NO_OCB_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_LAG3_NO_MRI_LESION, NO_LAG3_NO_MRI_LESION_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_LYMPH_CELLS_NO_AGE, NO_LYMPH_CELLS_NO_AGE_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_LYMPH_CELLS_NO_PMN_CELLS, NO_LYMPH_CELLS_NO_PMN_CELLS_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_LYMPH_CELLS_NO_OCB, NO_LYMPH_CELLS_NO_OCB_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_LYMPH_CELLS_NO_MRI_LESION, NO_LYMPH_CELLS_NO_MRI_LESION_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_TIGIT_NO_LAG3_NO_LYMPH_CELLS, NO_TIGIT_NO_LAG3_NO_LYMPH_CELLS_STR,df_experiments,df_michal_copy_transform)
df_experiments = experiments_RF_LR(df_livnat_all_patients,NO_TIGIT_NO_LAG3_NO_LYMPH_CELLS_NO_AGE, NO_TIGIT_NO_LAG3_NO_LYMPH_CELLS_NO_AGE_STR,df_experiments,df_michal_copy_transform)


df_experiments.to_excel('/content/gdrive/MyDrive/Colab Notebooks/experiments_excels/df_specificity_sensitivity_rmse_Livnat_only_all_patients_after_added_michal.xlsx',index=False)
df_experiments







,Model,Removed Features,Specificity,Sensitivity,RMSE
0,Random Forest Classifier,ALL FEATURES,0.897727,0.458333,
1,Logistic Regression,ALL FEATURES,0.931818,0.395833,
2,Random Forest Regressor,ALL FEATURES,,,1.71611
3,Random Forest Classifier,NO TIGIT,0.943182,0.5,
4,Logistic Regression,NO TIGIT,0.931818,0.395833,
5,Random Forest Regressor,NO TIGIT,,,1.70528
6,Random Forest Classifier,NO LAG3,0.909091,0.541667,
7,Logistic Regression,NO LAG3,0.931818,0.395833,
8,Random Forest Regressor,NO LAG3,,,1.74953
9,Random Forest Classifier,NO LYMPH CELLS,0.886364,0.5,


In [ ]:
df_michal_copy_transform.to_excel('/content/gdrive/MyDrive/Colab Notebooks/experiments_excels/df_michal_copy_transform_after_imputation.xlsx',index=False)


In [ ]:
df_michal.to_excel('/content/gdrive/MyDrive/Colab Notebooks/experiments_excels/df_michal_before_imputation.xlsx',index=False)


#Logistic Regression

filling missing data

In [ ]:
# num_med_Col = df['num_of_medicine']
# num_med_Col = num_med_Col[np.logical_not(np.isnan(num_med_Col))]
# df['num_of_medicine'].fillna(stats.median(num_med_Col), inplace=True)

In [ ]:
# df.to_csv('dfNoUK.csv',index=False)

In [ ]:
# #define numeric type diagnosis
# df['age_at_diagnosis'] = df['age_at_diagnosis'].astype('int64')
# # df['Cells'] = df['Cells'].astype('int64')
# # df['PMN_Cells'] = df['PMN_Cells'].astype('int64')
# # df['Lymph_Cells'] = df['Lymph_Cells'].astype('int64')
# df['num_of_medicine'] = df['num_of_medicine'].astype('int64')

# df['primary_EDSS_at_diagnosis'] = df['primary_EDSS_at_diagnosis'].astype('float64')
# df['1_years_EDSS_from_sample_date'] = df['1_years_EDSS_from_sample_date'].astype('float64')
# df['5_years_from_sample'] = df['5_years_from_sample'].astype('float64')
# df['EDSS_post_10_years'] = df['EDSS_post_10_years'].astype('float64')
# #define numeric type MRI
# df['MRI_lesion_mass'] = df['MRI_lesion_mass'].astype('float64')

# #define category type diagnosis
# df['diagnosis'] = df['diagnosis'].astype('category')
# df['oligoclonal_band'] = df['oligoclonal_band'].astype('category')
# df['sample_code'] = df['sample_code'].astype('category')
# df['gender'] = df['gender'].astype('category')


In [ ]:
# df.dtypes

In [ ]:
# mriLessionCol = df['MRI_lesion_mass']
# edssTenCol = df['primary_EDSS_at_diagnosis']
# edssTenCol = df['1_years_EDSS_from_sample_date']

# mriLessionCol = mriLessionCol[np.logical_not(np.isnan(mriLessionCol))]
# edssTenCol = edssTenCol[np.logical_not(np.isnan(edssTenCol))]
# edssTenCol = edssTenCol[np.logical_not(np.isnan(edssTenCol))]

# df['MRI_lesion_mass'].fillna(stats.median(mriLessionCol), inplace=True)
# df['primary_EDSS_at_diagnosis'].fillna(stats.median(edssTenCol), inplace=True)
# df['1_years_EDSS_from_sample_date'].fillna(stats.median(edssTenCol), inplace=True)


In [ ]:

# df['5_years_from_sample'].fillna(stats.median(df['5_years_from_sample']), inplace=True)
# df['EDSS_post_10_years'].fillna(stats.median(df['EDSS_post_10_years']), inplace=True)
# df['oligoclonal_band'].fillna(stats.mode(df['oligoclonal_band']), inplace=True)
# df['gender'].fillna(stats.mode(df['gender']), inplace=True)
# df['Lymph_Cells'].fillna(stats.median(df['Lymph_Cells']), inplace=True)
# df['Lymph_Cells'] = df['Lymph_Cells'].astype('int64')
# df['PMN_Cells'].fillna(stats.median(df['PMN_Cells']), inplace=True)
# df['PMN_Cells'] = df['PMN_Cells'].astype('int64')
# df['Cells'].fillna(stats.median(df['Cells']), inplace=True)
# df['Cells'] = df['Cells'].astype('int64')
